# CNN
This notebook is created to consider different examples of Neural Network architectures and understand the secret behind their efficiency.

## Residual Block
Let's consider two layers in a usual fully connected network. The connections are mainly governed by these equations:

1. $\begin{aligned} z^{[l+1]} = W^{[l+1]} \cdot a^{[l]} + b^{[l+1]}\end{aligned}$ 
2. $\begin{aligned} a^{[l+1]} = g(z^{[l+1]}) \end{aligned}$ where $g$ generally refers to the ***ReLU*** function
3. $\begin{aligned} z^{[l+2]} = W^{[l+2]} \cdot a^{[l+1]} + b^{[l+2]}\end{aligned}$ 
4. $\begin{aligned} a^{[l+2]} = g(z^{[l+2]}) \end{aligned}$ 

In theory a larger number of layers only helps the performance. Nevertheless, experiments found that starting from a certain threshhold, the training error starts to increase once again due to exploding and vinishing gradients. Therefore, the ***residual block*** was introduced.  
A Residual block is governed by a slightly different version of the system of equations above.
1. $\begin{aligned} z^{[l+1]} = W^{[l+1]} \cdot a^{[l]} + b^{[l+1]}\end{aligned}$ 
2. $\begin{aligned} a^{[l+1]} = g(z^{[l+1]}) \end{aligned}$ where $g$ generally refers to the ***ReLU*** function
3. $\begin{aligned} z^{[l+2]} = W^{[l+2]} \cdot a^{[l+1]} + b^{[l+2]}\end{aligned}$ 
4. $\begin{aligned} a^{[l+2]} = g(z^{[l+2] + a^{[l+1]}}) \end{aligned}$ 

Experiments proved that residual nets' training error is generally non-decreasing even with large number of hidden layers.

### Residual Block's intuition
Coupled with regularization (mainly L2), the parameters found in matrix $W^{[l]}$ generally shrink. Assuming the function $g$ is indeed the ***ReLu*** activation function, then $a^{[l]} \geq 0$ which means that $g(W^{[l+2]} \cdot a^{[l+1]} + b^{[l+1]} + a^{[l]}) \approx g(a^{[l]}) = a^{[l]}$ This is quite powerful, as
1. it is assured not to hurt performance
2. it might improve performance, if $W^{[l+2]}$ is not small enough, but not too small, some helpful features might be learnt from the data.

### 1 * 1 Convolution.
At first glance, the 1*1 might not be useful as it merely multiplies the values in a convolution by a real number. This is might be the case for layers $(n * n  * 1)$. Yet, 1 * 1  convolutions might be considered as networks inside the network. As each slice of the convolution layer: $(1 * 1 * n_{c})$ will be multipilied by a (1 * 1 * n_c) filter. Which is quite similar to the fully connected layers' functionality.

#### Applicataions
Assuming we have a $(n * n * n_c)$ convolution layer, We can shrink the width and height by using pooling layers. Yet, it might be necessary to shrink the number of channels. $(1 * 1 * n_c)$ filters are an efficient way to accomplish that. Of course, the 1 * 1 convolution layer might be added for the sole purpose of adding more complexity to the function learnt.


## Inseption Network
The choice of the number of layers is a fundamental choice in the network's architecture. The new concept introduced by the inception network is to try all the possible combinations. Instead of choosing either one among $(1 * 1), (3 * 3), (5 * 5)$ or a pooling layer, let's use every one of them and choose a uniform output shape (by padding) and stack the results. This picture taken from the CNN course by DeepLearningAI is a great [illustration](https://github.com/ayhem18/Towards_Data_science/blob/master/Machine_Learning/CNN/inceptionNet.png?raw=true). 
However, as promising and powerful as this idea seems, it turns out it it require significant computational power.
Let's solely consider a previous layer with dimensions $(28 * 28 * 192)$. Using a convolutional layer of dimensions $(5 * 5 *  32)$, a total of $(28 \cdot 28 \cdot 192 \cdot 5 \cdot 5 \cdot  32) \approx 120 \cdot 10 ^ 6$ operations take place.

### 1 * 1 Convolution in Inseption Networks
Instead of directly converting a $(28 * 28 * 192)$ convolution layer to a $(5 * 5 *  32)$ one, researchers suggested adding an intermediate (1 * 1) convolution layer, shrinking the initial layer to a resonably smaller layer, then apply the $(5 * 5 *  32)$ convolution. Let's consider the total number of operations with a $(1 * 1 * 16)$ intermediate or ***bottleneck*** layer: 
1. first operation produces a $28 * 28 * 16$ layer: $(28 \cdot 28 \cdot 192 \cdot 32) \approx 2.4 \cdot 10 ^ 6$
2. second operation produces the desired $(5 * 5 *  32):$ $(28 \cdot 28 \cdot 16 \cdot \cdot 28 \cdot 28 \cdot 32) \approx 10 \cdot 10 ^ 6$
The total operations are around $ \approx 12 \cdot 10 ^ 6$ which is around $10$ times smaller.
The idea of representation shrinkage is quite similar to Dimensional Reduction. As long as it is done within certain norms, the performance is unlikely to be hurt while saving significant computational power.

## MobileNet
The network architectures considered previously are significantly deep Networks and require a signficant computational power. The main motivation here is to apply the power of deep learning on machines with low computational power. Instead of using the usual normal convolution layer, we can use an alternative composed approach called ***depthwise + pointwise*** convolution
### DepthWise Convolution
Assuming we have a $(n_h, n_w, n_c)$ layer we will use only one filter $(f, f, n_c)$ to produce a $n_i, n_j, n_c$ intermediate output. The $i$-th chanel is the result of the $i$ th input channel the $i$i -th filter channel: a 2d simple convolution.

### Pointwise Convolution
The intermediate input volume is to be multiplied by $n_{filters}$ $(1 * 1 * n_c)$ convolutional layers. The result is a $n_i, n_j, n_f$ desired output layer.

#### Difference between Normal and Depth-Point wise convolution: output
In normal convolution, a cell at the $i$-th channel in the output is the result of the sum of channel wise convolution operation in the $i$ th filter. Each filter offers $n_c$ $(f, f)$ on which element wise convolution is applied on the $n_c$ input channels.  
In depth-point wise convolution, there is only on set of filter channels. Yet, the poitwise convolution offers a differnt linear combination of the result of convolution operations for each filter.

#### Difference between Normal and Depth-Point wise convolution: computation
In normal convolution, producing the output requires: $f ^ 2 \cdot n_c \cdot n_p \cdot n_f $.  
Depthwise convolution requires: $f ^ 2 \cdot n_p \cdot n_c$   
Pointwise convolution requires: $n_p \cdot n_c \cdot n_f$  
The ratio is computation is estimated as: $\frac{1}{f^2} + \frac{1}{n_f}$

Thus, MobileNet offers potential competitive performance as the normal convolution while requiring significantly less computational power.

### Inception Model
The inception network is built mainly on an inception module/component that can be described as follows:
1. The previous layer (or input data)
2. Each of the different convolutional layers preceeded by a 1 * 1 convolutional layer to shrink the input volume.
3. the output layer resulting from stucking the different output layer together.
The following image represents a good [example](https://th.bing.com/th/id/R.3310eee5ea970a4cdba43b5825b68f88?rik=H1sIVaLB0uPxPA&riu=http%3a%2f%2fmedia5.datahacker.rs%2f2018%2f11%2finception_module.png&ehk=ODgDxsM9fL13ezwSnEsWzlx3QiI%2ftTBHNUCi74S4jbY%3d&risl=&pid=ImgRaw&r=0).